# Get Code References from papers

This notebook:

 * Uses getpapers externally to download fulltext of all papers in EuPMC which contain github URLs
 * Textmines each paper fulltext and extract occurences of GitHub URLs
 * Outputs a data structure of the form: paper_DOI {{github_url: "http://github.com/blah/blah"}...}

In [21]:
import json

## Use getpapers to download fulltext of papers

We currently do this outside of the notebook, and assume that the files are available locally.

The command we are using is:

>getpapers --query 'github' -x --limit 100 -o data

which queries EuPMC for all papers containing the term 'github' and returns the full text of the first 100 papers matching this into the directory 'data'

## Textmine each paper

### File locations

In [6]:
# Directory containing the data
data_dir = '../data/'

# File containing the list of matching papers
matching_papers = data_dir + 'eupmc_fulltext_html_urls.txt'


In [19]:
# Get the list of subdirectories

papers = []

# For each line in matching_papers, strip the start of the line (http://europepmc.org/articles/)

with open(matching_papers, 'r') as f:
  for line in f:
     terms = line.split("/")
     papers.append(terms[-1].rstrip())


In [20]:
print(papers)

['PMC5802054', 'PMC5838108', 'PMC5833151', 'PMC5634325', 'PMC5832410', 'PMC5764482', 'PMC5819480', 'PMC5627421', 'PMC5753347', 'PMC5736641', 'PMC5707441', 'PMC5816961', 'PMC5796701', 'PMC5736239', 'PMC5667849', 'PMC5801530', 'PMC5768271', 'PMC5782805', 'PMC5838881', 'PMC5809091', 'PMC5775343', 'PMC5700758', 'PMC5804317', 'PMC5790920', 'PMC5815186', 'PMC5744633', 'PMC5801138', 'PMC5801807', 'PMC5746494', 'PMC5668921', 'PMC5754048', 'PMC5657068', 'PMC5828247', 'PMC5472659', 'PMC5794939', 'PMC5778627', 'PMC5655815', 'PMC5778497', 'PMC5813898', 'PMC5517642', 'PMC5508660', 'PMC5827353', 'PMC5807796', 'PMC5691210', 'PMC5654261', 'PMC5787250', 'PMC5697234', 'PMC5449644', 'PMC5628028', 'PMC5607581', 'PMC5713628', 'PMC5728484', 'PMC5634323', 'PMC5588604', 'PMC5814748', 'PMC5768164', 'PMC5706697', 'PMC5610716', 'PMC5712467', 'PMC5717846', 'PMC5817627', 'PMC5571441', 'PMC5751784', 'PMC5391935', 'PMC5756455', 'PMC5714127', 'PMC5552001', 'PMC5753187', 'PMC5821087', 'PMC5596339', 'PMC5530318', 'PMC5

In [26]:
# For each paper

for paper_dir in papers:

    # Read in the JSON file and get the DOI
    
    filename = data_dir + paper_dir + '/' + 'eupmc_result.json'
    print(filename)
    
    if filename:
        with open(filename, 'r') as f:
            paper_json = json.load(f)
        # Get the DOI
            paper_doi = paper_json['doi']
            print(paper_doi)
       
    
    # Read in the XML full text and mine for the github URLs

 

../data/PMC5802054/eupmc_result.json
['10.1186/s13059-017-1382-0']
../data/PMC5838108/eupmc_result.json
['10.1038/s41467-018-03297-7']
../data/PMC5833151/eupmc_result.json
['10.1186/s12859-018-2074-9']
../data/PMC5634325/eupmc_result.json
['10.1038/sdata.2017.151']
../data/PMC5832410/eupmc_result.json
['10.7554/elife.32822']
../data/PMC5764482/eupmc_result.json
['10.1371/journal.pone.0191298']
../data/PMC5819480/eupmc_result.json
['10.1038/sdata.2018.11']
../data/PMC5627421/eupmc_result.json
['10.1186/s13059-017-1319-7']
../data/PMC5753347/eupmc_result.json
['10.1093/nar/gkx1152']
../data/PMC5736641/eupmc_result.json
['10.1038/s41598-017-18257-2']
../data/PMC5707441/eupmc_result.json
['10.1002/pmic.201700244']
../data/PMC5816961/eupmc_result.json
['10.7717/peerj.4312']
../data/PMC5796701/eupmc_result.json
['10.1371/journal.pone.0192520']
../data/PMC5736239/eupmc_result.json
['10.1371/journal.pone.0189334']
../data/PMC5667849/eupmc_result.json
['10.1371/journal.pone.0187457']
../data/PM

FileNotFoundError: [Errno 2] No such file or directory: '../data//eupmc_result.json'

## Output data structure